In [41]:
import numpy as np
import pandas as pd
from scipy import stats

songs=pd.read_csv('data/tracks.csv')
songs=songs.head(100000)

In [42]:
songs_for_linear_regression = songs.copy()
songs_for_linear_regression.set_index('id',inplace=True)

In [43]:
def drop_columns(dataframe,to_be_deleted):
    dataframe.drop(to_be_deleted, axis=1, inplace=True)
to_be_deleted = ['id_artists', 'artists','name','release_date'] 

In [44]:
songs_for_linear_regression['release_date'] = pd.to_datetime(songs_for_linear_regression['release_date'], errors='coerce')

In [58]:
songs_for_linear_regression['year'] = pd.DatetimeIndex(songs_for_linear_regression['release_date']).year
songs_for_linear_regression['month'] = pd.DatetimeIndex(songs_for_linear_regression['release_date']).month


KeyError: 'release_date'

In [46]:
import sklearn
from sklearn import preprocessing

def scaler(dataframe,to_be_scaled):
    scaler = sklearn.preprocessing.StandardScaler(copy = True)
    dataframe[to_be_scaled] = scaler.fit_transform(dataframe[to_be_scaled].to_numpy())
# to_be_scaled = ['popularity','danceability','energy','key','loudness','speechiness','acousticness','instrumentalness','liveness','valence','tempo','time_signature']
to_be_scaled = ['loudness','duration_ms','tempo']

In [47]:
drop_columns(songs_for_linear_regression,to_be_deleted)#delete columns
songs_for_linear_regression["key"] = songs_for_linear_regression["key"].astype("category")
songs_for_linear_regression = pd.get_dummies(songs_for_linear_regression, columns=["key"])

In [48]:
songs_for_linear_regression["year"] = songs_for_linear_regression["year"].astype("category")
songs_for_linear_regression = pd.get_dummies(songs_for_linear_regression, columns=["year"])

In [ ]:
songs_for_linear_regression["month"] = songs_for_linear_regression["month"].astype("category")
songs_for_linear_regression = pd.get_dummies(songs_for_linear_regression, columns=["month"])

In [49]:
songs_for_linear_regression[(np.abs(stats.zscore(songs_for_linear_regression)) < 5).all(axis=1)]

In [50]:
scaler(songs_for_linear_regression,to_be_scaled)

In [51]:
songs_for_linear_regression

,popularity,duration_ms,explicit,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,...,year_2012,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018,year_2019,year_2020,year_2021
id,,,,,,,,,,,,,,,,,,,,,
35iwgR4jXetI318WEWsa1Q,6,-0.715036,0,0.645,0.4450,-0.292329,1,0.4510,0.674,0.744000,...,0,0,0,0,0,0,0,0,0,0
021ht4sdgPcrDgSk7JTbKY,0,-0.918905,0,0.695,0.2630,-1.878154,1,0.9570,0.797,0.000000,...,0,0,0,0,0,0,0,0,0,0
07A5yehtSnoedViJAZkNnc,0,-0.326256,0,0.434,0.1770,-1.705836,1,0.0512,0.994,0.021800,...,0,0,0,0,0,0,0,0,0,0
08FmqUhxtyLTn6pAh6bk45,0,-0.359873,0,0.321,0.0946,-2.928100,1,0.0504,0.995,0.918000,...,0,0,0,0,0,0,0,0,0,0
08y9GfoqCWfOGsKdwojr5e,0,-0.458082,0,0.402,0.1580,-0.934373,0,0.0390,0.989,0.130000,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3sFgcM3PVMvJ95n7mQJTU4,31,0.091859,0,0.817,0.7990,0.932639,0,0.1040,0.426,0.000000,...,0,0,0,0,0,0,0,0,0,0
4KBULZasCMBDKHY8IPUCML,32,1.905413,0,0.270,0.3690,-1.247285,1,0.0380,0.895,0.926000,...,0,0,0,0,0,0,0,0,0,0
6Vjk8MNXpQpi0F4BefdTyq,64,2.056594,0,0.483,0.6150,0.572863,1,0.0497,0.452,0.000414,...,0,0,0,0,0,0,0,0,0,0


In [52]:
songs_for_linear_regression.dropna(inplace=True)

# AND SO IT BEGINS

In [53]:
import statsmodels.api as sm

In [54]:
independent_variables = songs_for_linear_regression.columns.drop('valence')

In [55]:
y = songs_for_linear_regression['valence']
X = songs_for_linear_regression[independent_variables]

In [56]:
X = sm.add_constant(X)
model11 = sm.OLS(y, X).fit()
model11.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                valence   R-squared:                       0.522
Model:                            OLS   Adj. R-squared:                  0.521
Method:                 Least Squares   F-statistic:                     885.5
Date:                Thu, 10 Feb 2022   Prob (F-statistic):               0.00
Time:                        22:07:13   Log-Likelihood:                 29218.
No. Observations:              100000   AIC:                        -5.819e+04
Df Residuals:                   99876   BIC:                        -5.701e+04
Df Model:                         123                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -0.2056      0.005    -38.194      0.000      -0.216      -0.195
popularity          -0.0004   3.83e-05    -10.305      0.000      -0.000      -0.000
duration_ms         -0.0282      0.001    -47.850      0.000      -0.029      -0.027
explicit            -0.0736      0.003    -22.940      0.000      -0.080      -0.067
danceability         0.8551      0.004    221.002      0.000       0.847       0.863
energy               0.5213      0.005    110.326      0.000       0.512       0.531
loudness            -0.0160      0.001    -15.254      0.000      -0.018      -0.014
mode                 0.0155      0.001     12.077      0.000       0.013       0.018
speechiness         -0.1108      0.004    -26.647      0.000      -0.119      -0.103
acousticness         0.1221      0.003     42.555      0.000       0.116       0.128
instrumentalness    -0.0362      0.002    -17.513      0.000      -0.040      -0.032
liveness             0.0106      0.003      3.091      0.002       0.004       0.017
tempo                0.0319      0.001     53.404      0.000       0.031       0.033
time_signature       0.0029      0.001      2.563      0.010       0.001       0.005
key_0               -0.0166      0.002    -10.005      0.000      -0.020      -0.013
key_1               -0.0270      0.002    -12.999      0.000      -0.031      -0.023
key_2               -0.0149      0.002     -8.430      0.000      -0.018      -0.011
key_3               -0.0157      0.003     -6.067      0.000      -0.021      -0.011
key_4               -0.0212      0.002    -10.433      0.000      -0.025      -0.017
key_5               -0.0072      0.002     -3.938      0.000      -0.011      -0.004
key_6               -0.0255      0.002    -10.459      0.000      -0.030      -0.021
key_7               -0.0129      0.002     -7.626      0.000      -0.016      -0.010
key_8               -0.0219      0.002     -9.771      0.000      -0.026      -0.017
key_9               -0.0084      0.002     -4.692      0.000      -0.012      -0.005
key_10              -0.0145      0.002     -6.967      0.000      -0.019      -0.010
key_11              -0.0198      0.002     -8.653      0.000      -0.024      -0.015
year_1922            0.0583      0.015      3.940      0.000       0.029       0.087
year_1923            0.1458      0.008     18.067      0.000       0.130       0.162
year_1924           -0.0218      0.009     -2.446      0.014      -0.039      -0.004
year_1925            0.0924      0.007     13.471      0.000       0.079       0.106
year_1926           -0.0069      0.006     -1.171      0.242      -0.019       0.005
year_1927            0.1081      0.006     18.093      0.000       0.096       0.120
year_1928            0.0515      0.006      8.808      0.000       0.040       0.063
year_1929            0.

In [57]:
model11.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                valence   R-squared:                       0.522
Model:                            OLS   Adj. R-squared:                  0.521
Method:                 Least Squares   F-statistic:                     885.5
Date:                Thu, 10 Feb 2022   Prob (F-statistic):               0.00
Time:                        22:07:14   Log-Likelihood:                 29218.
No. Observations:              100000   AIC:                        -5.819e+04
Df Residuals:                   99876   BIC:                        -5.701e+04
Df Model:                         123                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -0.2056      0.005    -38.194      0.000      -0.216      -0.195
popularity          -0.0004   3.83e-05    -10.305      0.000      -0.000      -0.000
duration_ms         -0.0282      0.001    -47.850      0.000      -0.029      -0.027
explicit            -0.0736      0.003    -22.940      0.000      -0.080      -0.067
danceability         0.8551      0.004    221.002      0.000       0.847       0.863
energy               0.5213      0.005    110.326      0.000       0.512       0.531
loudness            -0.0160      0.001    -15.254      0.000      -0.018      -0.014
mode                 0.0155      0.001     12.077      0.000       0.013       0.018
speechiness         -0.1108      0.004    -26.647      0.000      -0.119      -0.103
acousticness         0.1221      0.003     42.555      0.000       0.116       0.128
instrumentalness    -0.0362      0.002    -17.513      0.000      -0.040      -0.032
liveness             0.0106      0.003      3.091      0.002       0.004       0.017
tempo                0.0319      0.001     53.404      0.000       0.031       0.033
time_signature       0.0029      0.001      2.563      0.010       0.001       0.005
key_0               -0.0166      0.002    -10.005      0.000      -0.020      -0.013
key_1               -0.0270      0.002    -12.999      0.000      -0.031      -0.023
key_2               -0.0149      0.002     -8.430      0.000      -0.018      -0.011
key_3               -0.0157      0.003     -6.067      0.000      -0.021      -0.011
key_4               -0.0212      0.002    -10.433      0.000      -0.025      -0.017
key_5               -0.0072      0.002     -3.938      0.000      -0.011      -0.004
key_6               -0.0255      0.002    -10.459      0.000      -0.030      -0.021
key_7               -0.0129      0.002     -7.626      0.000      -0.016      -0.010
key_8               -0.0219      0.002     -9.771      0.000      -0.026      -0.017
key_9               -0.0084      0.002     -4.692      0.000      -0.012      -0.005
key_10              -0.0145      0.002     -6.967      0.000      -0.019      -0.010
key_11              -0.0198      0.002     -8.653      0.000      -0.024      -0.015
year_1922            0.0583      0.015      3.940      0.000       0.029       0.087
year_1923            0.1458      0.008     18.067      0.000       0.130       0.162
year_1924           -0.0218      0.009     -2.446      0.014      -0.039      -0.004
year_1925            0.0924      0.007     13.471      0.000       0.079       0.106
year_1926           -0.0069      0.006     -1.171      0.242      -0.019       0.005
year_1927            0.1081      0.006     18.093      0.000       0.096       0.120
year_1928            0.0515      0.006      8.808      0.000       0.040       0.063
year_1929            0.